# Jacaranda street trees of LA
> This notebook reads and analyzes the locations of [jacaranda trees](https://en.wikipedia.org/wiki/Jacaranda_mimosifolia#:~:text=Jacaranda%20mimosifolia%20is%20a%20sub,poui%2C%20Nupur%20or%20fern%20tree.) along Los Angeles County streets and uploads them for use with Mapbox GL JS. 

---

#### Load Python tools and Jupyter config

In [23]:
import os
import json
import boto3
import mapbox
import requests
import mercantile
import pandas as pd
import jupyter_black
from io import BytesIO
import geopandas as gpd
import mapbox_vector_tile
from mapbox import Uploader

In [2]:
jupyter_black.load()
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
mapbox_key = os.environ.get("PERSONAL_MAPBOX_TILESET_ACCESS_TOKEN")

---

## Read

#### Read GeoJSON stored on S3

In [ ]:
# https://stiles-data.s3.us-west-1.amazonaws.com/trees/la_county_tree_locations.geojson

In [24]:
src = gpd.read_file("../data/processed/la_county_tree_locations.geojson").to_crs(
    "EPSG:3857"
)

#### Just the jacarandas

In [26]:
jac_gdf = src.query("jacaranda == True").reset_index(drop=True).copy()

#### How many trees?

In [27]:
len(jac_gdf)

41786

---

## Geography

#### LA County cities, unincorporated areas and LA City neighborhoods

In [28]:
hoods_gdf = gpd.read_file(
    "https://s3.us-west-1.amazonaws.com/stilesdata.com/la/la_city_hoods_county_munis.geojson"
).to_crs("EPSG:3857")

#### Clean up

In [29]:
hoods_gdf.columns = hoods_gdf.columns.str.lower()
hoods_gdf["coordinates"] = hoods_gdf.geometry.centroid
hoods_gdf["region_desc"] = (
    hoods_gdf.region.str.replace("-", " ").str.title().str.replace(" La", "")
)

#### Define the mapping from old type values to new descriptive values

In [30]:
type_mapping = {
    "standalone-city": "standalone city",
    "segment-of-a-city": "neighborhood in Los Angeles",
    "unincorporated-area": "unincorporated place in Los Angeles County",
}

#### Apply the mapping to the 'type_desc' column

In [31]:
hoods_gdf["type_desc"] = hoods_gdf["type"].map(type_mapping)

---

#### Merge hoods with trees

In [76]:
lahoods_merge = (gpd.sjoin(jac_gdf, hoods_gdf, predicate="within")).reset_index(
    drop=True
)[["id", "name", "type_desc", "city", "region_desc", "species", "geometry"]]

In [77]:
lahoods_merge["species"] = lahoods_merge["species"].str.lower()

In [78]:
lahoods_merge = lahoods_merge.to_crs(epsg=4326)
lahoods_merge["lat"] = lahoods_merge["geometry"].y
lahoods_merge["lon"] = lahoods_merge["geometry"].x

In [79]:
### Add a street view URL because Mapbox rejects lat/lon columns

In [80]:
# lahoods_merge["street_view_url"] = lahoods_merge.apply(
#     lambda row: f"http://maps.google.com/maps?q=&layer=c&cbll={row.lat},{row.lon}",
#     axis=1,
# )

#### Count how many jacarandas are in each place

In [81]:
lahoods_counts = (
    lahoods_merge.groupby("name")["geometry"]
    .count()
    .reset_index(name="jacaranda_count")
)

#### Merge that back to the geo file

In [82]:
jacs_hoods = lahoods_merge.merge(lahoods_counts, on="name")[
    ["name", "type_desc", "region_desc", "jacaranda_count", "geometry"]
].reset_index(drop=True)

---

## Exports

#### GeoJSON

In [83]:
lahoods_merge.drop(["id", "lat", "lon"], axis=1).to_file(
    "../data/processed/lacounty_jacaranda_locations.geojson",
    driver="GeoJSON",
)

In [84]:
lahoods_merge.head()

,id,name,type_desc,city,region_desc,species,geometry,lat,lon
0,3487,Long Beach,standalone city,None,Harbor,jacaranda,POINT (-118.17322 33.77729),33.777289,-118.173223
1,3586,West Hollywood,standalone city,None,Central,jacaranda,POINT (-118.36060 34.09029),34.090291,-118.360605
2,3587,West Hollywood,standalone city,None,Central,jacaranda,POINT (-118.38146 34.08060),34.080599,-118.381463
3,3588,West Hollywood,standalone city,None,Central,jacaranda,POINT (-118.38139 34.08046),34.080458,-118.381393
4,3868,Long Beach,standalone city,None,Harbor,jacaranda,POINT (-118.17422 33.84376),33.843760,-118.174220


#### Upload to Mapbox

In [86]:
# Replace with your Mapbox access token
access_token = mapbox_key
params = {"access_token": access_token}

# Request S3 credentials to stage file
r = requests.post("https://api.mapbox.com/uploads/v1/stiles/credentials", params=params)

try:
    r.raise_for_status()
except requests.exceptions.HTTPError as err:
    print(f"Error uploading the GeoJSON file.")
    raise

creds = r.json()

# Path to your GeoJSON file
geojson_file_path = "../data/processed/lacounty_jacaranda_locations.geojson"

# Remove the 'crs' member from the GeoJSON
with open(geojson_file_path, "r") as f:
    geojson_data = json.load(f)

if "crs" in geojson_data:
    del geojson_data["crs"]

with open(geojson_file_path, "w") as f:
    json.dump(geojson_data, f)

# Upload file to S3
with open(geojson_file_path, "rb") as f:
    s3_client = boto3.client(
        "s3",
        aws_access_key_id=creds["accessKeyId"],
        aws_secret_access_key=creds["secretAccessKey"],
        aws_session_token=creds["sessionToken"],
    )
    s3_client.upload_fileobj(f, creds["bucket"], creds["key"])
    print(
        f"Uploaded {geojson_file_path} to S3 bucket {creds['bucket']} with key {creds['key']}"
    )

# Generate Tileset
headers = {"Cache-Control": "no-cache"}
payload = {
    "url": creds["url"],
    "tileset": "stiles.jacaranda_tree_locations_v3",  # Use a new unique ID for the new tileset
    "name": "jacaranda_tree_locations",
}
s = requests.post(
    "https://api.mapbox.com/uploads/v1/stiles",
    params=params,
    headers=headers,
    json=payload,
)

try:
    s.raise_for_status()
except requests.exceptions.HTTPError as err:
    print(f"Error generating tileset.")
    raise

print("Tileset generation request sent successfully.")
print(s.json())

Uploaded ../data/processed/lacounty_jacaranda_locations.geojson to S3 bucket tilestream-tilesets-production with key 84/_pending/77rvr91y1ot12jofxytp0uwlc/stiles
Tileset generation request sent successfully.
{'id': 'clwu0q5858hij1vs26p1cdwoi', 'name': 'jacaranda_tree_locations', 'complete': False, 'error': None, 'created': '2024-05-31T01:41:26.163Z', 'modified': '2024-05-31T01:41:26.163Z', 'tileset': 'stiles.jacaranda_tree_locations_v3', 'owner': 'stiles', 'progress': 0}
